In [39]:
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd
import numpy as np
import html5lib
import requests
from openpyxl import Workbook

In [40]:
headers = {
    'User-Agent': 'OHA Investment Corp'
}
data = pd.read_excel(
    'URL to the excel', sheet_name='PSEC')

In [41]:
data

,cik,company_name,form_type,date_filed,file_name,web_url,html_link
0,1287032,PROSPECT ENERGY CORP,10-Q,11/12/2004,edgar/data/1287032/0000947871-04-002532.txt,https://www.sec.gov/Archives/edgar/data/128703...,https://www.sec.gov/Archives/edgar/data/128703...
1,1287032,PROSPECT ENERGY CORP,10-Q,2/10/2005,edgar/data/1287032/0000947871-05-000345.txt,https://www.sec.gov/Archives/edgar/data/128703...,https://www.sec.gov/Archives/edgar/data/128703...
2,1287032,PROSPECT ENERGY CORP,10-Q,5/13/2005,edgar/data/1287032/0000947871-05-000994.txt,https://www.sec.gov/Archives/edgar/data/128703...,https://www.sec.gov/Archives/edgar/data/128703...
3,1287032,PROSPECT ENERGY CORP,10-K,10/13/2005,edgar/data/1287032/0001206774-05-001638.txt,https://www.sec.gov/Archives/edgar/data/128703...,https://www.sec.gov/Archives/edgar/data/128703...
4,1287032,PROSPECT ENERGY CORP,10-Q,11/15/2005,edgar/data/1287032/0001206774-05-001821.txt,https://www.sec.gov/Archives/edgar/data/128703...,https://www.sec.gov/Archives/edgar/data/128703...
...,...,...,...,...,...,...,...
61,1287032,PROSPECT CAPITAL CORP,10-Q,11/6/2019,edgar/data/1287032/0001287032-19-000370.txt,https://www.sec.gov/Archives/edgar/data/128703...,https://www.sec.gov/Archives/edgar/data/128703...
62,1287032,PROSPECT CAPITAL CORP,10-Q,2/10/2020,edgar/data/1287032/0001287032-20-000043.txt,https://www.sec.gov/Archives/edgar/data/128703...,https://www.sec.gov/Archives/edgar/data/128703...
63,1287032,PROSPECT CAPITAL CORP,10-Q,5/11/2020,edgar/data/1287032/0001287032-20-000157.txt,https://www.sec.gov/Archives/edgar/data/128703...,https://www.sec.gov/Archives/edgar/data/128703...
64,1287032,PROSPECT CAPITAL CORP,10-K,8/26/2020,edgar/data/1287032/0001287032-20-000290.txt,https://www.sec.gov/Archives/edgar/data/128703...,https://www.sec.gov/Archives/edgar/data/128703...


In [42]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66 entries, 0 to 65
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   cik           66 non-null     int64 
 1   company_name  66 non-null     object
 2   form_type     66 non-null     object
 3   date_filed    66 non-null     object
 4   file_name     66 non-null     object
 5   web_url       66 non-null     object
 6   html_link     66 non-null     object
dtypes: int64(1), object(6)
memory usage: 3.7+ KB


In [43]:
def parse_and_trim(content, content_type):
    if content_type == 'HTML':
        soup = BeautifulSoup(content, 'html.parser')
    else:
        soup = BeautifulSoup(content, 'html.parser')

    for tag in soup.recursiveChildGenerator():
        try:
            tag.attrs = None
        except AttributeError:
            pass

    for linebreak in soup.find_all('br'):
        linebreak.extract()

    return soup

In [44]:
def extract_tables(soup_content, qtr_date):
    master_table = None
    all_tags = soup_content.find_all(True)
    print(type(all_tags))
    for tag in soup_content.find_all(text=re.compile('^.*CONSOLIDATED\s+SCHEDULE(S|)\s+OF\s+INVESTMENTS.*$')):
        date_str = re.search(r'([A-Za-z]+) (\d{1,2}), (\d{4})', tag)
        if date_str is None:
            next_line = tag.find_next(text=re.compile(
                r'([A-Za-z]+) (\d{1,2}), (\d{4})')).text
            date_str = re.search(r'([A-Za-z]+) (\d{1,2}), (\d{4})', next_line)
        if date_str is None:
            next_line = tag.next.next.next.next.next.next.text
            date_str = re.search(r'([A-Za-z]+) (\d{1,2}), (\d{4})', next_line)
        if date_str is not None:
            date_str = str(date_str.group())
            date_str = unicodedata.normalize('NFKD', date_str)
            if qtr_date.replace(',', '').strip().lower() in date_str.replace(',', '').strip().lower():
                print('Table found: ')
                html_table = tag.find_next('table')
                if master_table is None:
                    master_table = pd.read_html(
                        html_table.prettify(), skiprows=0, flavor='bs4')[0]
                    master_table = master_table.applymap(lambda x: unicodedata.normalize(
                        'NFKD', x.strip().strip(u'\u200b').replace('—', '-')) if type(x) == str else x)
                    master_table = master_table.replace(r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan,
                                                                                              regex=True)
                    master_table = master_table.dropna(how='all', axis=0)
                else:
                    new_table = pd.read_html(
                        html_table.prettify(), skiprows=0, flavor='bs4')[0]
                    new_table = new_table.applymap(lambda x: unicodedata.normalize(
                        'NFKD', x.strip().strip(u'\u200b').replace('—', '-')) if type(x) == str else x)
                    new_table = new_table.replace(r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan,
                                                                                        regex=True)
                    new_table = new_table.dropna(how='all', axis=0)
                    # print('head')
                    # print(new_table.head()) # text
                    master_table = master_table.append(
                        new_table.dropna(how='all', axis=0).reset_index(
                            drop=True).drop(index=0),
                        ignore_index=True)

    master_table = master_table.applymap(
        lambda x: x.strip().strip(u'\u200b') if type(x) == str else x)
    master_table = master_table.replace(r'^\s*$', np.nan, regex=True).replace(
        r'^\s*\$\s*$', np.nan, regex=True).replace(r'^\s*\)\s*$', np.nan, regex=True)
    return master_table

In [45]:
# this one is the real one

def process_table(soi_table_df, append_str):
    soi_table_df = soi_table_df.replace(r'^\s*\$\s*$', np.nan, regex=True)
    soi_table_df = soi_table_df.dropna(how='all', axis=1)
    soi_table_df = soi_table_df.dropna(
        how='all', axis=0).reset_index(drop=True)

    # Separate header and data
    soi_table_header = soi_table_df.iloc[1].dropna(how='any')
    print('header: ')
    print(soi_table_header)
    soi_table_data_df = soi_table_df.iloc[2:]
    print('1: ' + str(soi_table_data_df.shape))

    # Drop Full NnN rows
    soi_table_data_df = soi_table_data_df.dropna(how='all', axis=1)
    soi_table_data_df = soi_table_data_df.dropna(
        how='all', axis=0).reset_index(drop=True)
    print('2: ' + str(soi_table_data_df.shape))

    # Rename columns to integer range
    num_cols = soi_table_data_df.shape[1]
    data_col_mapper = dict(zip(soi_table_data_df.columns.to_list(), [
                           i for i in range(0, num_cols)]))
    soi_table_data_df = soi_table_data_df.rename(columns=data_col_mapper)
    print('3: ' + str(soi_table_data_df.shape))
    soi_table_data_df.to_csv('csv_file/3_'+append_str+'.csv')

    # Drop "Control Instruments" rows
    # soi_table_data_df = soi_table_data_df.dropna(subset=[i for i in range(1, num_cols)], how='all')
    # print('4: ' + str(soi_table_data_df.shape))

    # if num_cols > 7:
    #    # Drop labeled subtotal/total rows
    #    soi_table_data_df = soi_table_data_df.dropna(subset=[1, 3, 5, 6, 7], how='all')
    #    print('5: ' + str(soi_table_data_df.shape))

    # Drop labeled subtotal rows
    soi_table_data_df = soi_table_data_df.dropna(
        subset=[i for i in range(1, num_cols - 2)], how='all')
    print('5: ' + str(soi_table_data_df.shape))

    # Drop subtotal/total rows based on regex
    sub_total_filter_pattern = r'^([Ss]ubtotal)|([Tt]otal)'
    sub_total_filter = soi_table_data_df[0].str.contains(
        sub_total_filter_pattern).replace(np.NaN, False)
    print(sub_total_filter)
    soi_table_data_df = soi_table_data_df[~sub_total_filter]

    # Drop Full NnN rows/cols
    soi_table_data_df = soi_table_data_df.dropna(how='all', axis=1)
    soi_table_data_df = soi_table_data_df.dropna(
        how='all', axis=0).reset_index(drop=True)
    print('6: ' + str(soi_table_data_df.shape))

    # Rename columns to integer range
    num_cols = soi_table_data_df.shape[1]
    data_col_mapper = dict(zip(soi_table_data_df.columns.to_list(), [
                           i for i in range(0, num_cols)]))
    soi_table_data_df = soi_table_data_df.rename(columns=data_col_mapper)

    # Drop totals rows
    soi_table_data_df = soi_table_data_df.dropna(
        subset=[i for i in range(0, num_cols-2)], how='all')
    print('7: ' + str(soi_table_data_df.shape))

    # Forward Fill first 2 columns
    ffill_cols = [i for i in range(0, num_cols-4)]
    soi_table_data_df[ffill_cols] = soi_table_data_df[ffill_cols].fillna(
        method='ffill')
    print('8: ' + str(soi_table_data_df.shape))
    soi_table_data_df.to_csv('csv_file/8_'+append_str+'.csv')

    # Drop rows with only first 2/3 columns present
    soi_table_data_df = soi_table_data_df.dropna(
        subset=[i for i in range(num_cols-4, num_cols)], how='all')
    print('9: ' + str(soi_table_data_df.shape))

    # Fill data cols NaN with 0
    soi_table_data_df = soi_table_data_df.fillna(0)
    soi_table_data_df = soi_table_data_df.replace('—', 0)
    print('10: ' + str(soi_table_data_df.shape))
    soi_table_data_df.to_csv('csv_file/10_'+append_str+'.csv', index=False)

    # Replace hyphen with 0
    soi_table_data_df = soi_table_data_df.replace('-', 0, regex=False)
    soi_table_data_df = soi_table_data_df.replace('%', "", regex=False)

    # Typecast data cols to int
    for col_index in range(num_cols-3, num_cols):
        col_name = soi_table_data_df.columns[col_index]
        soi_table_data_df[col_name] = pd.to_numeric(
            soi_table_data_df[col_name], errors='coerce')

# Multiply numeric columns by 1000
    numeric_cols = [
        col_name for col_name in soi_table_data_df.columns[num_cols-3:num_cols]]
    soi_table_data_df[numeric_cols] = soi_table_data_df[numeric_cols].apply(
        lambda x: x * 1000)

    print('11: ' + str(soi_table_data_df.shape))

    # first_str = soi_table_data_df[3].iloc[0]
    # print(first_str)
    # ord_l = [ord(c) for c in first_str]

    # print(ord_l)
    # Rename columns to table headers
    header_col_mapper = dict(
        zip(soi_table_data_df.columns.to_list(), soi_table_header))
    soi_table_data_df = soi_table_data_df.rename(columns=header_col_mapper)
    print('12: ' + str(soi_table_data_df.shape))

    soi_table_data_df = soi_table_data_df.iloc[:, :soi_table_header.shape[0]]

    return soi_table_data_df

In [46]:
# def process_table(soi_table_df, append_str):
#     soi_table_df = soi_table_df.replace(r'^\s*\$\s*$', np.nan, regex=True)
#     soi_table_df = soi_table_df.dropna(how='all', axis=1)
#     soi_table_df = soi_table_df.dropna(
#         how='all', axis=0).reset_index(drop=True)
#     soi_table_header = soi_table_df.iloc[1].dropna(how='any')
#     print('header: ')
#     print((soi_table_header.shape[0]))
#     soi_table_df = soi_table_df.iloc[:, :soi_table_header.shape[0]]
#     soi_table_data_df = soi_table_df.iloc[2:]
#     print('1: ' + str(soi_table_data_df.shape))

#     # Drop Full NnN rows
#     soi_table_data_df = soi_table_data_df.dropna(how='all', axis=1)
#     soi_table_data_df = soi_table_data_df.dropna(
#         how='all', axis=0).reset_index(drop=True)
#     print('2: ' + str(soi_table_data_df.shape))

#     # Rename columns to integer range
#     num_cols = soi_table_data_df.shape[1]
#     data_col_mapper = dict(zip(soi_table_data_df.columns.to_list(), [
#                            i for i in range(0, num_cols)]))
#     soi_table_data_df = soi_table_data_df.rename(columns=data_col_mapper)
#     print('3: ' + str(soi_table_data_df.shape))
#     soi_table_data_df.to_csv('csv_file/3_'+append_str+'.csv')

#     # Drop "Control Instruments" rows
#     soi_table_data_df = soi_table_data_df.dropna(
#         subset=[i for i in range(1, num_cols)], how='all')
#     print('4: ' + str(soi_table_data_df.shape))

#     if num_cols > 7:
#         # Drop labeled subtotal/total rows
#         soi_table_data_df = soi_table_data_df.dropna(
#             subset=[1, 3, 5, 6, 7], how='all')
#         print('5: ' + str(soi_table_data_df.shape))

#     # Drop labeled subtotal rows
#     soi_table_data_df = soi_table_data_df.dropna(
#         subset=[i for i in range(1, num_cols - 2)], how='all')
#     print('5: ' + str(soi_table_data_df.shape))

#     # Drop subtotal/total rows based on regex
#     sub_total_filter_pattern = r'^([Ss]ubtotal)|([Tt]otal)'
#     sub_total_filter = soi_table_data_df[0].str.contains(
#         sub_total_filter_pattern).replace(np.NaN, False)
#     print(sub_total_filter)
#     soi_table_data_df = soi_table_data_df[~sub_total_filter]

#     # Drop Full NnN rows/cols
#     soi_table_data_df = soi_table_data_df.dropna(how='all', axis=1)
#     soi_table_data_df = soi_table_data_df.dropna(
#         how='all', axis=0).reset_index(drop=True)
#     print('6: ' + str(soi_table_data_df.shape))

#     # Rename columns to integer range
#     num_cols = soi_table_data_df.shape[1]
#     data_col_mapper = dict(zip(soi_table_data_df.columns.to_list(), [
#                            i for i in range(0, num_cols)]))
#     soi_table_data_df = soi_table_data_df.rename(columns=data_col_mapper)
#     soi_table_data_df = soi_table_data_df.dropna(
#         subset=[i for i in range(0, num_cols - 2)], how='all')
#     print('7: ' + str(soi_table_data_df.shape))

#     # Drop rows with only first 2/3 columns present
#     soi_table_data_df = soi_table_data_df.dropna(
#         subset=[i for i in range(num_cols - 4, num_cols)], how='all')
#     print('9: ' + str(soi_table_data_df.shape))

#     # Fill data cols NaN with 0
#     soi_table_data_df = soi_table_data_df.fillna(0)
#     soi_table_data_df = soi_table_data_df.replace('—', 0)
#     print('10: ' + str(soi_table_data_df.shape))
#     soi_table_data_df.to_csv('csv_file/10_'+append_str+'.csv', index=False)
#     # Replace hyphen with 0
#     soi_table_data_df = soi_table_data_df.replace('-', 0, regex=False)
#     soi_table_data_df = soi_table_data_df.replace('%', '', regex=False)

#     for col_index in range(num_cols - 3, num_cols):
#         col_name = soi_table_data_df.columns[col_index]
#         soi_table_data_df[col_name] = pd.to_numeric(
#             soi_table_data_df[col_name], errors='coerce')
#     numeric_cols = [
#         col_name for col_name in soi_table_data_df.columns[num_cols - 3:num_cols]]
#     soi_table_data_df[numeric_cols] = soi_table_data_df[numeric_cols].apply(
#         lambda x: x * 1000)

#     # Rename columns to table headers
#     header_col_mapper = dict(
#         zip(soi_table_data_df.columns.to_list(), soi_table_header))
#     soi_table_data_df = soi_table_data_df.rename(columns=header_col_mapper)
#     print('12: ' + str(soi_table_data_df.shape))

#     return soi_table_data_df

In [47]:
def remove_multiple_spaces(string):
    pattern = r'\s+'
    replaced_string = re.sub(pattern, ' ', string)
    return replaced_string


def find_qrt_date(content):
    qtr_date = content.find_all(text=re.compile(
        r'for\s+(the\s+)?(fiscal\s+)?year\s+ended\s+|for\s+the\s+quarter\s+ended\s+|for\s+the\s+quarterly\s+period\s+ended\s+', re.IGNORECASE))
    qtr_match = re.search(
        r'([A-Za-z]+)\s+(\d{1,2}),\s+(\d{4})', qtr_date[0].replace('\n', ''))
    if qtr_match is None:
        qtr_match = qtr_match = re.search(
            r'([A-Za-z]+) (\d{1,2}), (\d{4})', qtr_date[1])
    return remove_multiple_spaces(str(qtr_match.group()))

In [48]:
qtr_dates = []
for index, url in enumerate(data['html_link']):
    response = requests.get(url, headers=headers)
    content = parse_and_trim(response.content, 'HTML')
    qtr_date = find_qrt_date(content)
    print(qtr_date)
    qtr_dates.append(qtr_date.replace(',', '').strip())
data['qtr_date'] = qtr_dates

September 30, 2004
December 31, 2004
March 31, 2005
June 30, 2005
September 30, 2005
September 30, 2005
December 31, 2005
March 31, 2006
June 30, 2006
September 30, 2006
December 31, 2006
March 31, 2007
June 30, 2007
September 30, 2007
December 31, 2007
March 31, 2008
JUNE 30, 2008
September 30, 2008
December 31, 2008
March 31, 2009
JUNE 30, 2009
September 30, 2009
December 31, 2009
March 31, 2010
June 30, 2010
September 30, 2010
December 31, 2010
March 31, 2011
June 30, 2011
September 30, 2011
December 31, 2011
March 31, 2012
June 30, 2012
September 30, 2012
December 31, 2012
March 31, 2013
June 30, 2013
September 30, 2013
December 31, 2013
March 31, 2014
June 30, 2014
September 30, 2014
December 31, 2014
March 31, 2015
June 30, 2015
September 30, 2015
December 31, 2015
March 31, 2016
June 30, 2016
September 30, 2016
December 31, 2016
March 31, 2017
June 30, 2017
September 30, 2017
December 31, 2017
March 31, 2018
June 30, 2018
September 30, 2018
December 31, 2018
March 31, 2019
June 

In [49]:
data.head()

,cik,company_name,form_type,date_filed,file_name,web_url,html_link,qtr_date
0,1287032,PROSPECT ENERGY CORP,10-Q,11/12/2004,edgar/data/1287032/0000947871-04-002532.txt,https://www.sec.gov/Archives/edgar/data/128703...,https://www.sec.gov/Archives/edgar/data/128703...,September 30 2004
1,1287032,PROSPECT ENERGY CORP,10-Q,2/10/2005,edgar/data/1287032/0000947871-05-000345.txt,https://www.sec.gov/Archives/edgar/data/128703...,https://www.sec.gov/Archives/edgar/data/128703...,December 31 2004
2,1287032,PROSPECT ENERGY CORP,10-Q,5/13/2005,edgar/data/1287032/0000947871-05-000994.txt,https://www.sec.gov/Archives/edgar/data/128703...,https://www.sec.gov/Archives/edgar/data/128703...,March 31 2005
3,1287032,PROSPECT ENERGY CORP,10-K,10/13/2005,edgar/data/1287032/0001206774-05-001638.txt,https://www.sec.gov/Archives/edgar/data/128703...,https://www.sec.gov/Archives/edgar/data/128703...,June 30 2005
4,1287032,PROSPECT ENERGY CORP,10-Q,11/15/2005,edgar/data/1287032/0001206774-05-001821.txt,https://www.sec.gov/Archives/edgar/data/128703...,https://www.sec.gov/Archives/edgar/data/128703...,September 30 2005


In [50]:
data['qtr_date'] = pd.to_datetime(data['qtr_date'])
data = data[data['qtr_date'] >= pd.to_datetime('2013-03-31')]

In [51]:
data.shape

(31, 8)

In [52]:
# count = 0
# for qtr_date, html_link in (list(zip(data['qtr_date'], data['html_link'])):
#     print('start')
#     response = requests.get(html_link, headers=headers)
#     content = parse_and_trim(response.content, 'HTML')
#     print('content DONE')
#     date = pd.to_datetime(qtr_date).date()
#     formatted_date = date.strftime("%B %d, %Y")
#     print(formatted_date)
#     master_table = extract_tables(content, formatted_date)
#     print(count, "master_table DONE")
#     count += 1
#     print("-------------------------------------------------------------------------------------------------------------------")

In [53]:
data = data.drop(36)
data

,cik,company_name,form_type,date_filed,file_name,web_url,html_link,qtr_date
35,1287032,PROSPECT CAPITAL CORP,10-Q,5/6/2013,edgar/data/1287032/0001104659-13-037639.txt,https://www.sec.gov/Archives/edgar/data/128703...,https://www.sec.gov/Archives/edgar/data/128703...,2013-03-31
37,1287032,PROSPECT CAPITAL CORP,10-Q,11/4/2013,edgar/data/1287032/0001104659-13-080552.txt,https://www.sec.gov/Archives/edgar/data/128703...,https://www.sec.gov/Archives/edgar/data/128703...,2013-09-30
38,1287032,PROSPECT CAPITAL CORP,10-Q,2/3/2014,edgar/data/1287032/0001104659-14-005997.txt,https://www.sec.gov/Archives/edgar/data/128703...,https://www.sec.gov/Archives/edgar/data/128703...,2013-12-31
39,1287032,PROSPECT CAPITAL CORP,10-Q,5/6/2014,edgar/data/1287032/0001287032-14-000120.txt,https://www.sec.gov/Archives/edgar/data/128703...,https://www.sec.gov/Archives/edgar/data/128703...,2014-03-31
40,1287032,PROSPECT CAPITAL CORP,10-K,8/25/2014,edgar/data/1287032/0001287032-14-000128.txt,https://www.sec.gov/Archives/edgar/data/128703...,https://www.sec.gov/Archives/edgar/data/128703...,2014-06-30
41,1287032,PROSPECT CAPITAL CORP,10-Q,11/6/2014,edgar/data/1287032/0001287032-14-000176.txt,https://www.sec.gov/Archives/edgar/data/128703...,https://www.sec.gov/Archives/edgar/data/128703...,2014-09-30
42,1287032,PROSPECT CAPITAL CORP,10-Q,2/4/2015,edgar/data/1287032/0001287032-15-000032.txt,https://www.sec.gov/Archives/edgar/data/128703...,https://www.sec.gov/Archives/edgar/data/128703...,2014-12-31
43,1287032,PROSPECT CAPITAL CORP,10-Q,5/6/2015,edgar/data/1287032/0001287032-15-000127.txt,https://www.sec.gov/Archives/edgar/data/128703...,https://www.sec.gov/Archives/edgar/data/128703...,2015-03-31
44,1287032,PROSPECT CAPITAL CORP,10-K,8/26/2015,edgar/data/1287032/0001287032-15-000229.txt,https://www.sec.gov/Archives/edgar/data/128703...,https://www.sec.gov/Archives/edgar/data/128703...,2015-06-30
45,1287032,PROSPECT CAPITAL CORP,10-Q,11/4/2015,edgar/data/1287032/0001287032-15-000322.txt,https://www.sec.gov/Archives/edgar/data/128703...,https://www.sec.gov/Archives/edgar/data/128703...,2015-09-30


In [54]:
path = 'URL to the out put excel'
writer = pd.ExcelWriter(path, engine='openpyxl')

In [55]:
count = 0
for qtr_date, html_link in zip(data['qtr_date'], data['html_link']):
    print('start')
    response = requests.get(html_link, headers=headers)
    content = parse_and_trim(response.content, 'HTML')
    print('content DONE')
    date = pd.to_datetime(qtr_date).date()
    formatted_date = date.strftime("%B %d, %Y")
    print(formatted_date)
    master_table = extract_tables(content, formatted_date)
    print(count, "master_table DONE")
    processed_table_ = process_table(
        master_table, formatted_date.replace(' ', '').replace(',', ''))
    print(processed_table_)
    processed_table_.to_excel(
        writer, sheet_name=formatted_date.replace(' ', '').replace(',', ''), index=False)
    print(count, "processed_table_ DONE")
    count += 1
    print("-------------------------------------------------------------------------------------------------------------------")
writer.save()
writer.close()

start
content DONE
March 31, 2013
<class 'bs4.element.ResultSet'>
Table found: 
Table found: 


/var/folders/mf/yrkcqqr56t955_zz9p9f4swc0000gn/T/ipykernel_41360/209566738.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_table = master_table.append(
/var/folders/mf/yrkcqqr56t955_zz9p9f4swc0000gn/T/ipykernel_41360/209566738.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_table = master_table.append(
/var/folders/mf/yrkcqqr56t955_zz9p9f4swc0000gn/T/ipykernel_41360/209566738.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_table = master_table.append(


Table found: 
Table found: 
Table found: 


/var/folders/mf/yrkcqqr56t955_zz9p9f4swc0000gn/T/ipykernel_41360/209566738.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_table = master_table.append(
/var/folders/mf/yrkcqqr56t955_zz9p9f4swc0000gn/T/ipykernel_41360/209566738.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_table = master_table.append(
/var/folders/mf/yrkcqqr56t955_zz9p9f4swc0000gn/T/ipykernel_41360/209566738.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_table = master_table.append(
/var/folders/mf/yrkcqqr56t955_zz9p9f4swc0000gn/T/ipykernel_41360/209566738.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_table = master_ta

Table found: 
Table found: 
Table found: 
Table found: 


...

Table found: 
Table found: 
Table found: 
Table found: 
Table found: 
Table found: 
Table found: 
Table found: 
Table found: 


/var/folders/mf/yrkcqqr56t955_zz9p9f4swc0000gn/T/ipykernel_41360/209566738.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_table = master_table.append(
/var/folders/mf/yrkcqqr56t955_zz9p9f4swc0000gn/T/ipykernel_41360/209566738.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_table = master_table.append(
/var/folders/mf/yrkcqqr56t955_zz9p9f4swc0000gn/T/ipykernel_41360/209566738.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_table = master_table.append(
/var/folders/mf/yrkcqqr56t955_zz9p9f4swc0000gn/T/ipykernel_41360/209566738.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_table = master_ta

Table found: 
Table found: 
Table found: 


/var/folders/mf/yrkcqqr56t955_zz9p9f4swc0000gn/T/ipykernel_41360/209566738.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_table = master_table.append(
/var/folders/mf/yrkcqqr56t955_zz9p9f4swc0000gn/T/ipykernel_41360/209566738.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_table = master_table.append(
/var/folders/mf/yrkcqqr56t955_zz9p9f4swc0000gn/T/ipykernel_41360/209566738.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_table = master_table.append(
/var/folders/mf/yrkcqqr56t955_zz9p9f4swc0000gn/T/ipykernel_41360/209566738.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_table = master_ta

Table found: 
Table found: 
Table found: 
Table found: 
Table found: 
Table found: 
Table found: 
Table found: 
Table found: 
29 master_table DONE
header: 
0        Portfolio Company
1                 Industry
2       Investments(1)(37)
3     Acquisition Date(44)
4             Coupon/Yield
5                    Floor
6           Legal Maturity
7          Principal Value
8           Amortized Cost
9           Fair  Value(2)
10         % of Net Assets
Name: 1, dtype: object
1: (530, 24)
2: (530, 24)
3: (530, 24)
5: (516, 24)
1      False
2      False
3      False
4      False
5      False
       ...  
525    False
526    False
527    False
528    False
529    False
Name: 0, Length: 516, dtype: bool
6: (501, 24)
7: (501, 24)
8: (501, 24)
9: (4, 24)
10: (4, 24)
11: (4, 24)
12: (4, 24)
                 Portfolio Company Industry  Investments(1)(37)  \
331                   Distributors        0                   0   
335        Air Freight & Logistics        0                   0   
392  Div